Stats data from https://datafinder.stats.govt.nz/layer/104430-meshblock-2020-centroid-inside/

In [1]:
import pandas as pd
import shapely
from shapely import Polygon
from shapely import wkt
import dash_leaflet as dl

locales_df = pd.read_csv("data\\nz_suburbs_and_localities.csv")
ece_services_df = pd.read_csv("data\\ece_services.csv")

In [2]:
#drop services with bad latitude or longitude data
ece_services_df = ece_services_df.drop(ece_services_df[ece_services_df['Latitude'] > 90].index)
ece_services_df = ece_services_df[ece_services_df['Latitude'].notna()]
ece_services_df = ece_services_df[ece_services_df['Longitude'].notna()]

#filter ece services to just columns needed
ece_services_df = ece_services_df[['Service Name', 'Latitude', 'Longitude', 'Town / City', 'General Electorate', 'Street', 'Suburb', 'Town / City', 'Territorial Authority', 'Service Type', '20 Hours ECE', 'Equity Index', 'Max. Licenced Positions', 'Under 2\'s']]

In [3]:
#tidy up territorial authorities
ece_services_df['Territorial Authority'].replace(['Auckland.*'], 'Auckland', regex=True, inplace=True)

In [32]:
from shapely.geometry import Point, Polygon
import shapely.wkt

def ece_capacity(locale_pop, locale_ta, locale_poly):
    ece_capacity = 0
    ece_services = []

    if (pd.isna(locale_pop)):
        return(0)
    else:

        #get services just for this ta
        ece_services_ta_df = ece_services_df[ece_services_df.apply(lambda x: x['Territorial Authority'] in locale_ta, axis=1)]

        polygon = shapely.wkt.loads(locale_poly)


        # if (len(ece_services_ta_df) == 0):
        #     print(locale_ta + " returned " + str(len(ece_services_ta_df)) + " services.")

        for item, row in ece_services_ta_df.iterrows():

            point = Point(row['Longitude'], row['Latitude'])

            if (point.within(polygon)):
                ece_capacity = ece_capacity + row['Max. Licenced Positions']
                ece_services.append({'name': row['Service Name'], 'lat' : row['Latitude'], 'lng' : row['Longitude'], 'max' : row['Max. Licenced Positions']})

        return([ece_capacity, ece_services])

In [33]:
def ece_demand(population, ece_profile):
    prop_U5 = 0.059
    prop_att = 0.607
    

    if (pd.isna(population)):
        return(0)
    else:

        U5_pop = round(population * prop_U5)
        ece_pop = round(U5_pop * prop_att)

        if (ece_profile[0] == 0):
            ece_capacity = 0
        else:
            ece_capacity = round(ece_pop/ece_profile[0], 2)

        result = {
            'U5_pop' : U5_pop,
            'ece_pop' : ece_pop,
            'ece_places' : ece_profile[0],
            'ece_services' : ece_profile[1],
            'ece_capacity' : ece_capacity        
        }

    return (result)

In [34]:
#identifies if the ta is covered by the locale
def identify_ta(locale_ta, target_tas):
    
    for ta in target_tas:
        if ta in locale_ta:
            return (True)
    
    return (False)

In [35]:
locales_df["ece_capacity"] = locales_df.apply(lambda row: ece_capacity(row['population_estimate'], row['territorial_authority_ascii'],row['WKT']), axis =1)

In [36]:
locales_df["ece_demand"] = locales_df.apply(lambda row: ece_demand(row['population_estimate'], row['ece_capacity']), axis =1)

In [37]:
locales_df['include'] = locales_df.apply(lambda row: identify_ta(row['territorial_authority_ascii'], ['Christchurch City', 'Selwyn District', 'Waimakariri']), axis=1)

In [38]:
locales_df.drop(['ece_capacity'], inplace=True, axis=1)

In [39]:
locales_df = pd.concat([locales_df.drop('ece_demand', axis=1),  pd.json_normalize(locales_df['ece_demand'])], axis=1)

In [40]:
locales_df = locales_df[['WKT', 'id', 'name', 'U5_pop', 'ece_pop', 'ece_places', 'ece_services', 'ece_capacity', 'include']]

In [41]:
def get_poly_coordinates(poly):

    #create geometry view
    polygon =  wkt.loads(poly)
    polygon = shapely.geometry.mapping(polygon)

    positions = polygon['coordinates']
    positions = [list(elem) for elem in list(positions[0])]

    for item in positions:
        item.reverse()

    return (positions)

In [42]:
locales_df['positions'] = locales_df.apply(lambda row: get_poly_coordinates(row['WKT']), axis =1)

In [43]:
locales_df

,WKT,id,name,U5_pop,ece_pop,ece_places,ece_services,ece_capacity,christchurch_city,positions
0,"POLYGON ((171.355078005649 -42.070367999046,17...",7140,Meybille Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-42.070367999046, 171.355078005649], [-42.07..."
1,"POLYGON ((172.879629996903 -43.8992019989129,1...",7828,Whakamoa Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-43.8992019989129, 172.879629996903], [-43.8..."
2,"POLYGON ((174.985845000394 -36.8789809991999,1...",7545,Sunkist Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-36.8789809991999, 174.985845000394], [-36.8..."
3,"POLYGON ((174.882553784236 -37.7972490227562,1...",7072,Lorenzen Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-37.7972490227562, 174.882553784236], [-37.7..."
4,"POLYGON ((175.086644006051 -36.8384020053753,1...",8442,Deadmans Bay,NaN,NaN,NaN,NaN,NaN,False,"[[-36.8384020053753, 175.086644006051], [-36.8..."
...,...,...,...,...,...,...,...,...,...,...
6551,"POLYGON ((168.351876007319 -46.4009030017748,1...",70,Avenal,79.0,48.0,392.0,"[{'name': 'Active Explorers Invercargill', 'la...",0.12,False,"[[-46.4009030017748, 168.351876007319], [-46.4..."
6552,"POLYGON ((170.503520992814 -45.8631389967932,1...",8799,North Dunedin,587.0,356.0,247.0,"[{'name': 'Kelsey-Yaralla Kindergarten', 'lat'...",1.44,False,"[[-45.8631389967932, 170.503520992814], [-45.8..."
6553,"POLYGON ((171.246843006687 -44.384725000598,17...",8764,Maori Hill,68.0,41.0,0.0,[],0.00,False,"[[-44.384725000598, 171.246843006687], [-44.38..."
6554,"POLYGON ((174.817220235269 -41.3437231286051,1...",942,Moa Point,2.0,1.0,0.0,[],0.00,False,"[[-41.3437231286051, 174.817220235269], [-41.3..."


In [16]:
import pandas as pd
import geopandas as gpd

# Load your DataFrame (replace 'your_data.csv' with your file path)
df = pd.read_csv("data\\ece_demand.csv")

# Assuming you have a column named 'wkt_polygon' containing WKT data
# Convert the 'wkt_polygon' column to a GeoSeries
df['geometry'] = gpd.GeoSeries.from_wkt(df['WKT'])


In [17]:
# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [18]:
gdf = gdf[gdf['include'] == True]

In [19]:
gdf = gdf[['name', 'ece_services', 'ece_capacity', 'geometry']]

In [21]:
# Save the GeoDataFrame to a GeoJSON file (replace 'output.geojson' with your desired file path)
gdf.to_file('output.geojson', driver='GeoJSON')

print("GeoJSON file saved successfully.")

GeoJSON file saved successfully.


In [1]:
import pandas as pd

ece_services_df = pd.read_csv("data\\ece_services_pop.csv")

ece_services_df = ece_services_df.loc[:, ece_services_df.columns != 'Unnamed: 0']

ece_demand = pd.read_csv("data\\ece_demand.csv")

In [2]:
ece_demand.head()

,Unnamed: 0,WKT,id,name,U5_pop,ece_pop,ece_places,ece_services,ece_capacity,include,geometry,positions
0,0,"POLYGON ((171.355078005649 -42.070367999046,17...",7140,Meybille Bay,NaN,NaN,NaN,NaN,NaN,False,"POLYGON ((171.355078005649 -42.070367999046, 1...","[[-42.070367999046, 171.355078005649], [-42.07..."
1,1,"POLYGON ((172.879629996903 -43.8992019989129,1...",7828,Whakamoa Bay,NaN,NaN,NaN,NaN,NaN,False,"POLYGON ((172.879629996903 -43.8992019989129, ...","[[-43.8992019989129, 172.879629996903], [-43.8..."
2,2,"POLYGON ((174.985845000394 -36.8789809991999,1...",7545,Sunkist Bay,NaN,NaN,NaN,NaN,NaN,False,"POLYGON ((174.985845000394 -36.8789809991999, ...","[[-36.8789809991999, 174.985845000394], [-36.8..."
3,3,"POLYGON ((174.882553784236 -37.7972490227562,1...",7072,Lorenzen Bay,NaN,NaN,NaN,NaN,NaN,False,"POLYGON ((174.882553784236 -37.7972490227562, ...","[[-37.7972490227562, 174.882553784236], [-37.7..."
4,4,"POLYGON ((175.086644006051 -36.8384020053753,1...",8442,Deadmans Bay,NaN,NaN,NaN,NaN,NaN,False,"POLYGON ((175.086644006051 -36.8384020053753, ...","[[-36.8384020053753, 175.086644006051], [-36.8..."


In [1]:
!rm -f ./.git/index.lock

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
def get_locale(lat, lng, ta):

    #filter to the locales that share the same ta as the service
    locales_df_ta = locales_df[locales_df.apply(lambda x: x['territorial_authority'].contains(ta), axis=1)]

    for item, row in locales_df_ta.iterrows():
        if 

In [18]:
ece_services_df

,Unnamed: 0,Service Name,Latitude,Longitude,Town / City,General Electorate,Street,Suburb,Town / City.1,Service Type,20 Hours ECE,Equity Index,Max. Licenced Positions,Under 2's,mesh,pop
0,0,Anne West Kindergarten,-35.118228,173.258565,Kaitaia,Northland,92 Pukepoto Road,NaN,Kaitaia,Free Kindergarten,Yes,EQI 2,30.0,0.0,"[{'mesh_id': 2304, 'lat': -35.04590938929111, ...",4611.0
1,1,Doubtless Bay Kindergarten,-34.994478,173.464730,Taipa,Northland,5 E Mamaru Road,NaN,Taipa,Free Kindergarten,Yes,EQI 3,30.0,0.0,"[{'mesh_id': 3000, 'lat': -35.01701480355184, ...",2697.0
2,2,Mission Place Kindergarten,-35.116080,173.270685,Kaitaia,Northland,Mission Place,NaN,Kaitaia,Free Kindergarten,Yes,EQI 3,30.0,0.0,"[{'mesh_id': 2306, 'lat': -35.06739771868577, ...",4572.0
3,3,Hillcrest Kindergarten,-35.405553,173.796409,Kaikohe,Northland,Cnr Park & Hillcrest Roads,NaN,Kaikohe,Free Kindergarten,Yes,EQI 2,30.0,0.0,"[{'mesh_id': 32501, 'lat': -35.32682545029647,...",3201.0
4,4,Waitangi Kindergarten,-35.278121,174.081808,Paihia,Northland,Te Kemara Avenue,NaN,Paihia,Free Kindergarten,Yes,EQI 4,30.0,0.0,"[{'mesh_id': 35700, 'lat': -35.31276312175539,...",5532.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,4471,Te Kōhanga Reo I Ngā Hau E Whā,-46.424321,168.353016,Invercargill,Invercargill,98 Balmoral Drive,NaN,Invercargill,Te Kōhanga Reo,Yes,EQI 3,40.0,15.0,"[{'mesh_id': 3117000, 'lat': -46.4014926804934...",47739.0
4449,4472,Te Kōhanga Reo o Murihiku,-46.430588,168.387830,Invercargill,Invercargill,Skye Street,NaN,Invercargill,Te Kōhanga Reo,Yes,EQI 5+,10.0,4.0,"[{'mesh_id': 3117000, 'lat': -46.4014926804934...",47730.0
4450,4473,Te Kōhanga Reo o Tūmanako Rāwhiti,-46.455293,168.370376,Invercargill,Invercargill,28 Ottrey Street,NaN,Invercargill,Te Kōhanga Reo,Yes,EQI 1,16.0,6.0,"[{'mesh_id': 3117000, 'lat': -46.4014926804934...",46551.0
4451,4474,Te Kōhanga Reo o Te Rākau Kowhai o Ngā Tamariki,-46.385365,168.346139,Invercargill,Invercargill,18 Willis Street,NaN,Invercargill,Te Kōhanga Reo,Yes,EQI 2,30.0,12.0,"[{'mesh_id': 3117000, 'lat': -46.4014926804934...",49260.0


In [4]:
import geopandas as gpd
from shapely.geometry import Point

ece_services_df['geometry'] = ece_services_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

In [21]:
gdf = gpd.GeoDataFrame(ece_services_df, geometry='geometry')

In [22]:
gdf.to_file('services.geojson', driver='GeoJSON')